### A very basic and quick pipeline to derive conservatively called consensus fasta seqeunce from Hyb-Seq data

Requirements: 

    Trimmomatic-0.36  
    bwa mem 
    samtools  
    bcftools  
    mafft 
    trimal
    amas  
    FastTree  
    

Raw reads are trimmed using standard trimmomatic settings, aligned to the baits using BWA mem and a vcf file derived from the bam.
The vcf is filtered for quality of at least 30 and to obly include homozygous alternate allele SNPS

Raw reads should in the folder:

    ~/Process/Hairdrier/
    
The expected file name format is:

    SAMPLE1_1.fastq.gz, SAMPLE1_2.fastq.gz
    
The BWA index of the baits should be prepared with the following command:
    
    bwa index Baits.fna

Consensus fastas for each bait for each sample can be produced by making a list of the samples to use and running the following while loop:  
    
    while read f ; do ./raw_to_fasta_BWA.sh "$f" ; done < sample_list
    

This produces a multi fasta file for each sample with the form:
    >Bait_name1
    TCTCACCTACGAATTGATAATGCTGTTCGATGCAATAGCGTTCTCCACTTTTTCAGCATA
    ACTCAACGGTCCACAATGTTTATAAAAATAAAAAATAAAAATAACAAACAGTGACTTGAC
    >Bait_name2
    TAGGAACTTGAGTCGAATCAAGAAAAATCTCGCTTTATCATACGAAGAAGCACCTATCCA
    TACCAAAAATAATAAAGTTGGGAAAAAGTTCATTACAAATGGTCCGGGAAACATATCTCG
    
    
This needs to be converted to a set of multifastas for each bait using the script Switch_multi_fasta.py.  Make a folder called By_locus and two test files - one with the list of samples, one with the list of bait names.

    mkdir By_locus/
    ls *_consensus.fna > fasta_files
    grep ">" SAMPLE_consensus.fna | sed 's/>//g' > locus_list
    
    python3 Switch_multi_fasta.py

This generates many empty files in your working folder which need to be removed with:

    find . -size 0 -delete
    
The by-locus fasta files are in By_locus/ with the form:

BAIT_NAME.fasta

\>Sample1.fast_BAIT_NAME
TTATCCTCCTTCCCACAGCACACTCATTGACTTCTGTCTCGTGATGCCGTGCTACTCGTTGAAACCTGCTACTGTAGGTACATGAACATGTCCTGAAGCGGCGACATCTCATTTCCACTCTGAAAATGGCTTTCCATCGTCGCATCCATGTAGTAGTTATAAGGAAATTGCAGCGATTTCTCCAATTCCTTCCATTTGGGCTCGCTCTGCACTT
\>Sample2.fast_BAIT_NAME
TTATCCTCCTTCCCACAGCACACTCATTGACTTCTGTCTCGTGATGCCGTGCTACTCGTTGAAACCTGCTACTGTAGGTACATGAACATGTCCTGAAGCGGCGACATCTCATTTCCACTCTGAAAATGGCTTTCCATCGTCGCATCCATGTAGTAGTTATAAGGAAATTGCAGCGATTTCTCCAATTCCTTCCATTTGGGCT

The fasta headers of these files have the locus name in.  If you want to concatenate these you will want to remove this, leaving just the accession name:

    while read f ; do sed ’s/‘\.fast_"$f"’//g’ "$f".fasta > "$f".fna ; done < ../locus_list
    
\>Sample1
TTATCCTCCTTCCCACAGCACACTCATTGACTTCTGTCTCGTGATGCCGTGCTACTCGTTGAAACCTGCTACTGTAGGTACATGAACATGTCCTGAAGCGGCGACATCTCATTTCCACTCTGAAAATGGCTTTCCATCGTCGCATCCATGTAGTAGTTATAAGGAAATTGCAGCGATTTCTCCAATTCCTTCCA
\>Sample2
TTATCCTCCTTCCCACAGCACACTCATTGACTTCTGTCTCGTGATGCCGGGCTACTCGTAGAAACCTGCTACTGTAGGTACATGAACATGTCCTGGAGCGGCGACATCTCATTTCCACTTTGAAAATGGCTTTCCATCGTCGTATCCATGTAGTAGTTATAAGGAAATTGCAGTGATTTCTCCCATTCCTTCCATTTGGGCTCGCTCTGCACTTC

The multifastas can be individually aligned using mafft:

    while read f ; do mafft --thread 4 "$f".fna > "$f"_mafft.fna ; done < locus_list
  
Strict-trimming of the alignments:  
        
    while read f ; do trimal -in "$f"_mafft.fna -out "$f"_mafft_trim.fna -fasta -strict ; done < locus_list 
        
Summarising each alignment:

    while read f ; do python3 ~/amas-0.93/amas/AMAS.py -f fasta -d dna -i "$f"_mafft_trim.fna -s -o "$f".summary.txt ; done < ../locus_list

The alignments can be concatenated and a quick fast-tree alignment doen to check the assembly before further analysis
The lengths of the alignments are put into the header by trimal.  These need removing to allow concatenation:

    while read f ; do sed 's/ [0-9]* bp//g' "$f"_mafft_trim.fna > "$f".fasta ; done < ../locus_list

Concatenating using AMAS.  Put all the alignemed an trimmed fastas into a new folder.  Move into that folder and call AMAS:

    python3 ~/amas-0.93/amas/AMAS.py -f fasta -d dna -i *.fasta -c


Quick FastTree for checking:

    fasttree -gtr -nt  < concat.fasta > concat.tre  
